In [1]:
import pandas as pd 
import numpy as np
import os 
import json
from model.banco import Banco
from sqlalchemy import create_engine
from dotenv import load_dotenv

In [2]:
with open('listas.json', 'r') as file:
    LISTAS = json.load(file)
LISTAS_DATA = LISTAS['listas']

In [3]:
load_dotenv()
DB_URL = os.getenv('DB_URL')

In [4]:
QUERY = """
SELECT
	s.id AS submission_id,
	s.hitPercentage,
	s.char_change_number,
	s.timeConsuming,
	s.createdAt,
	q.id AS question_id,
	q.title AS question_title,
	lq.id AS list_id,
	u.id AS user_id,
	chu.enrollment AS user_enrollment,
	c.id AS class_id,
	c.name AS class_name
FROM
	submission s 
JOIN 
	question q ON s.question_id = q.id
JOIN
	user u ON s.user_id = u.id 
JOIN
	class c ON s.class_id = c.id
JOIN
	classhasuser chu ON c.id = chu.class_id AND u.id = chu.user_id
JOIN
	listquestions lq ON lq.id = s.listQuestions_id  
WHERE 
	c.year > 2021 AND
	lq.id IN (
		'4f3452ae-87d4-4a5f-9532-c7634da2cff0',
		'70e6c800-7583-485d-ac9f-9c42c48dd785',
		'42d5c4ad-7b44-4e4e-b2e5-2997785132c8',
		'dbfcc83c-f014-44ae-8260-f06d4e1370ba',
		'70d1aa7e-00cb-4428-892f-9ac67cc50449',
		'f5b1d992-68de-4854-97d8-c4bd84bd593b',
		'dd1cd083-6c00-4605-9ff1-163bd4069363',
		'0a2c49a2-482c-40d7-bc44-ee368f089560',
		'aca84206-87b2-4478-afa4-6193119cc06a',
		'34691c68-f01d-46ad-808d-6bbd44bcd9a2',
		'813dedaa-0989-489b-9804-3850ae4b83d1',
		'f5bc770d-2ac7-4060-ab1c-e42b5541f7a7',
		'93cd33c4-109a-401e-87e9-e26ff5929d37',
		'c4d09d1b-73a5-45c6-8a0d-ead669114b65',
		'1551e3ed-c875-4bb3-8e4e-58407df40776'
	);
"""
consulta = Banco(DB_URL)
result = consulta.select(QUERY)
del consulta

In [5]:
def calculate_metrics_for_list(list_id):
    list_data = result[result['list_id'] == list_id]
    metrics = list_data.groupby(['user_id', 'class_id']).agg(
        submissao=('submission_id', 'count'),
        numero_de_questoes=('question_id', 'nunique'),
        acertos=('hitPercentage', lambda x: (x == 100).sum()),
        media_submissoes=('submission_id', 'count'),
        totalmente_erradas=('hitPercentage', lambda x: (x == 0).sum()),
        parcialmente_erradas=('hitPercentage', lambda x: ((x > 0) & (x < 100)).sum()),
        totalmente_certas=('hitPercentage', lambda x: (x == 100).sum()),
        tempo_total_gasto=('timeConsuming', 'sum'),
        tempo_medio_gasto=('timeConsuming', 'mean'),
        tempo_desvio_padrao=('timeConsuming', 'std'),
        percentual_questoes_certas=('hitPercentage', 'mean'),
        sum_char_by_list=('char_change_number', 'sum'),
        mean_char_by_list=('char_change_number', 'mean'),
        std_char_by_list=('char_change_number', 'std'),
        submeteu=('submission_id', 'count')
    )
    metrics['submeteu'] = metrics['submeteu'] > 0
    return metrics

list_metrics_dfs = []
for i, lista in enumerate(LISTAS_DATA, start=1):
    for list_name, list_id in lista.items():
        metrics_df = calculate_metrics_for_list(list_id)
        metrics_df.columns = [f'{col}_list_id{i:02d}' for col in metrics_df.columns]
        list_metrics_dfs.append(metrics_df)

final_metrics_df = pd.concat(list_metrics_dfs, axis=1)
final_metrics_df.to_csv('metrics.csv')

In [16]:
QUERY = """
select 
    c.id as class_id,
    c.year as year,
    c.semester as semester,
    ch.enrollment as Matrícula,
    ch.user_id as user_id
from 
    class c
join 
    classhasuser ch
where year >= 2020 and year < 2024
"""
consulta = Banco(DB_URL)
result = consulta.select(QUERY)
del consulta

In [20]:
result = result.drop_duplicates(subset=['year', 'semester', 'Matrícula', 'user_id'], keep='first')
result.reset_index(drop=True, inplace=True)

In [21]:
result.to_csv('alunos_classes.csv',index=False)